In [2]:
import numpy as np
np.random.seed(2017)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import to_categorical
from keras import backend as K

In [3]:
batch_size = 128
nb_classes = 5
nb_epoch = 5

# input image dimensions
img_rows, img_cols = 28, 28

# number of convolutional filters to use
nb_filters = 32

# size of pooling area for max pooling
pool_size = 2

# convolution kernel size
kernel_size = 3

input_shape = (img_rows, img_cols, 1)

In [4]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
X_test_gte5 = X_test[y_test >= 5]         # np_utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# Train model for digits 0 to 4
def train_model(model, train, test, nb_classes):
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = to_categorical(train[1], nb_classes)
    Y_test = to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    model.fit(X_train, Y_train,
              batch_size=batch_size, epochs=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [9]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  padding='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model, (X_train_lt5, y_train_lt5), (X_test_lt5, y_test_lt5), nb_classes)

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 1s 4ms/step - loss: 1.6078 - accuracy: 0.2120 - val_loss: 1.6043 - val_accuracy: 0.2024
Epoch 2/5
240/240 [==============================] - 1s 3ms/step - loss: 1.6069 - accuracy: 0.2126 - val_loss: 1.6033 - val_accuracy: 0.2032
Epoch 3/5
240/240 [==============================] - 1s 3ms/step - loss: 1.6057 - accuracy: 0.2196 - val_loss: 1.6024 - val_accuracy: 0.2039
Epoch 4/5
240/240 [==============================] - 1s 3ms/step - loss: 1.6046 - accuracy: 0.2228 - val_loss: 1.6014 - val_accuracy: 0.2047
Epoch 5/5
240/240 [==============================] - 1s 3ms/step - loss: 1.6039 - accuracy: 0.2243 - val_loss: 1.6004 - val_accuracy: 0.2068
Test score: 1.6004316806793213
Test accuracy: 0.2068495750427246


***Transfer existing trained model on 0 to 4 to build model for digits 5 to 9***

In [10]:
# freeze feature layers and rebuild model
for layer in feature_layers:
    layer.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model, (X_train_gte5, y_train_gte5), (X_test_gte5, y_test_gte5), nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 1s 2ms/step - loss: 1.6080 - accuracy: 0.2248 - val_loss: 1.6065 - val_accuracy: 0.2119
Epoch 2/5
230/230 [==============================] - 0s 1ms/step - loss: 1.6077 - accuracy: 0.2244 - val_loss: 1.6063 - val_accuracy: 0.2119
Epoch 3/5
230/230 [==============================] - 0s 1ms/step - loss: 1.6076 - accuracy: 0.2215 - val_loss: 1.6062 - val_accuracy: 0.2125
Epoch 4/5
230/230 [==============================] - 0s 1ms/step - loss: 1.6077 - accuracy: 0.2234 - val_loss: 1.6060 - val_accuracy: 0.2125
Epoch 5/5
230/230 [==============================] - 0s 1ms/step - loss: 1.6073 - accuracy: 0.2258 - val_loss: 1.6058 - val_accuracy: 0.2125
Test score: 1.6058093309402466
Test accuracy: 0.2125077098608017
